In [1]:
import csv
import numpy as np
import pandas as pd
import json
import pickle

from main import train_setup
import sklearn.metrics as skmetric

In [2]:
with open('/raid/xiaoyuz1/goemotions/goemotions/data/emotions.txt', 'r') as f:
    emotion_list = f.readlines()
    emotion_list = [s.strip() for s in emotion_list]

emotion_text_to_label = dict(zip(emotion_list, range(len(emotion_list))))
label_to_emotion_text = dict(zip(range(len(emotion_list)), emotion_list))

ekman_to_go = json.load(open('/raid/xiaoyuz1/goemotions/goemotions/data/ekman_mapping.json'))
go_to_ekman = {'neutral' : 'neutral'}
for k,v in ekman_to_go.items():
    for vi in v:
        go_to_ekman[vi] = k
ekman_texts = list(ekman_to_go.keys())
ekman_to_idx = dict(zip(ekman_texts, range(len(ekman_texts))))
ekman_to_idx["neutral"] = len(ekman_texts)


sentiment_to_go = json.load(open('/raid/xiaoyuz1/goemotions/goemotions/data/sentiment_mapping.json'))
go_to_sentiment = {'neutral' : 'neutral'}
for k,v in sentiment_to_go.items():
    for vi in v:
        go_to_sentiment[vi] = k
sentiment_texts = list(sentiment_to_go.keys())
sentiment_to_idx = dict(zip(sentiment_texts, range(len(sentiment_texts))))
sentiment_to_idx["neutral"] = len(sentiment_texts)

In [3]:
label_to_emotion_text[25]

'sadness'

In [4]:
SEEDS = [
    79719,30010,46921,25577,52538,56440,41228,66558,48642,69556
]

In [5]:
meta_fname_template = "/raid/xiaoyuz1/goemotions/save_path/bert_seed-{}-meta.pkl"
pkl_fname_template = "/raid/xiaoyuz1/goemotions/pred_result/baseline/test_pred_seed-{}_label.pkl"

In [6]:
def individual_emotion_result(gt, pred, emotion_list, label2idx=None, text2idx=None):
    precs = []
    recalls = []
    f1s = []

    for emotion_label, emotion_text in enumerate(emotion_list):
        if label2idx is not None:
            emotion_idx = label2idx[emotion_label]
        else:
            assert text2idx is not None
            emotion_idx = text2idx[emotion_text]
            
        y_true = np.asarray(gt)
        y_pred = np.asarray(pred)

        y_true = y_true[:,emotion_idx]
        y_pred = y_pred[:,emotion_idx]

        prec = skmetric.precision_score(y_true, y_pred, average="binary")
        recall = skmetric.recall_score(y_true, y_pred, average="binary")
        f1 = skmetric.f1_score(y_true, y_pred, average="binary")

        precs += [prec]
        recalls += [recall]
        f1s += [f1]

    df_table_4 = pd.DataFrame({'Emotion': emotion_list,
                       'Precision': precs,
                       'Recall': recalls,
                      'F1' : f1s})
    
    return df_table_4

In [7]:
def all_individual_emotion_results(emotion_list, pkl_fname_template, meta_fname_template, seeds):
    df_tables = []
    for seed in seeds: 
        fh = open(pkl_fname_template.format(seed), 'rb')
        gt, pred, _ = pickle.load(fh)
        
        fh2 = open(meta_fname_template.format(seed), "rb")
        meta = pickle.load(fh2)
        
        label2idx = meta['labels2idxes'][0][0]
        label2idx = {int(key) : int(value) for key, value in label2idx.items()}
        idx2label = {value: key for key, value in label2idx.items()}
        
        df_table = individual_emotion_result(gt, pred, emotion_list, label2idx=label2idx)
        df_tables.append(df_table)
    
    emo_precs = []
    emo_recalls = []
    emo_f1s = []
    for i in range(len(emotion_list)):
        precs = []
        recalls = []
        f1s = []
        
        for df_table in df_tables:
            precs += [df_table.iloc[i]['Precision']]
            recalls += [df_table.iloc[i]['Recall']]
            f1s += [df_table.iloc[i]['F1']]
        
        prec_avg, prec_std = np.mean(precs), np.std(precs)
        recall_avg, recall_std = np.mean(recalls), np.std(recalls)
        f1_avg, f1_std = np.mean(f1s), np.std(f1s)

        emo_precs += [prec_avg]
        emo_recalls += [recall_avg]
        emo_f1s += [f1_avg]
        
#     df_emo = pd.DataFrame({'Emotion': emotion_list,
#                        'Precision': emo_precs,
#                        'Recall': emo_recalls,
#                       'F1' : emo_f1s})
    
    return {'Emotion': emotion_list,
                       'Precision': emo_precs,
                       'Recall': emo_recalls,
                      'F1' : emo_f1s}

In [8]:
emo_dict = all_individual_emotion_results(
    emotion_list, pkl_fname_template, meta_fname_template, SEEDS)

In [9]:
emo_ekman = []
emo_sentiment = []
for emo in emotion_list:
    emo_ekman += [go_to_ekman[emo]]
    emo_sentiment += [go_to_sentiment[emo]]

emo_dict['Ekman'] = emo_ekman
emo_dict['Sentiment'] = emo_sentiment

df_emo = pd.DataFrame(emo_dict)

In [10]:
df_emo.sort_values(by=['F1'], ascending=False)

,Emotion,Precision,Recall,F1,Ekman,Sentiment
15,gratitude,0.910913,0.904545,0.907614,joy,positive
1,amusement,0.764272,0.862500,0.810303,joy,positive
18,love,0.736025,0.825630,0.778093,joy,positive
0,admiration,0.650005,0.681548,0.664809,joy,positive
14,fear,0.629763,0.666667,0.647096,fear,negative
24,remorse,0.553345,0.741071,0.632717,sadness,negative
27,neutral,0.646696,0.611360,0.628417,neutral,neutral
17,joy,0.579339,0.601863,0.589668,joy,positive
25,sadness,0.568826,0.533974,0.550376,sadness,negative
20,optimism,0.577842,0.522043,0.548263,joy,positive


In [11]:
df_emo[df_emo['Sentiment'] == 'negative'].sort_values(by=["F1"], ascending=False)

,Emotion,Precision,Recall,F1,Ekman,Sentiment
14,fear,0.629763,0.666667,0.647096,fear,negative
24,remorse,0.553345,0.741071,0.632717,sadness,negative
25,sadness,0.568826,0.533974,0.550376,sadness,negative
2,anger,0.534629,0.449495,0.487735,anger,negative
11,disgust,0.507208,0.433333,0.466085,disgust,negative
12,embarrassment,0.498073,0.381081,0.428977,sadness,negative
10,disapproval,0.389205,0.346067,0.365863,anger,negative
19,nervousness,0.339894,0.365217,0.350755,fear,negative
3,annoyance,0.343905,0.320937,0.331786,anger,negative
16,grief,0.386667,0.250000,0.296313,sadness,negative


In [95]:
myseed = 48642
meta_fname = meta_fname_template.format(myseed)
pkl_fname = pkl_fname_template.format(myseed)

with open(meta_fname, "rb") as f:
    meta = pickle.load(f)

label2idx = meta['labels2idxes'][0][0]
label2idx = {int(key) : int(value) for key, value in label2idx.items()}
idx2label = {value: key for key, value in label2idx.items()}

df_test = pd.read_csv("/raid/xiaoyuz1/goemotions/goemotions/data/test.csv")

fh = open(pkl_fname, 'rb')
gt, pred, _ = pickle.load(fh)
gt = np.asarray(gt)
pred = np.asarray(pred)

In [109]:
import copy

neutral_idx = label2idx[emotion_text_to_label['neutral']]

pred_ideal = copy.deepcopy(pred)
for row_idx,(gt_row, pred_row) in enumerate(zip(gt, pred)):
    gt_idxs = list(np.where(gt_row)[0])
    
    if(neutral_idx in gt_idxs):
        continue
    
    if(pred_row[neutral_idx] == 1):
        for j in gt_idxs:
            pred_ideal[row_idx, j] = 1
        pred_ideal[row_idx, neutral_idx] = 0

In [111]:
df_ideal = individual_emotion_result(gt, pred_ideal, emotion_list, label2idx=label2idx)

In [112]:
df_ideal.sort_values(by=['F1'], ascending=False)

,Emotion,Precision,Recall,F1
15,gratitude,0.932551,0.903409,0.917749
1,amusement,0.755556,0.901515,0.822107
18,love,0.729537,0.861345,0.789981
27,neutral,1.000000,0.623391,0.768011
14,fear,0.662921,0.756410,0.706587
0,admiration,0.662478,0.732143,0.695570
20,optimism,0.656410,0.688172,0.671916
17,joy,0.628571,0.683230,0.654762
2,anger,0.637255,0.656566,0.646766
26,surprise,0.661654,0.624113,0.642336


In [117]:
df_ideal[df_ideal.Emotion.isin([
    'fear',
  'nervousness',
  'remorse',
  'embarrassment',
  'disappointment',
  'sadness',
  'grief',
  'disgust',
  'anger',
  'annoyance',
  'disapproval',
    'neutral',
])].sort_values(by=["F1"], ascending=False)

,Emotion,Precision,Recall,F1
27,neutral,1.000000,0.623391,0.768011
14,fear,0.662921,0.756410,0.706587
2,anger,0.637255,0.656566,0.646766
25,sadness,0.631250,0.647436,0.639241
16,grief,0.571429,0.666667,0.615385
24,remorse,0.533333,0.714286,0.610687
10,disapproval,0.565217,0.632959,0.597173
11,disgust,0.572581,0.577236,0.574899
12,embarrassment,0.642857,0.486486,0.553846
3,annoyance,0.493438,0.587500,0.536377


In [21]:

def probe(gt, pred, row_i, study_emo):
    print(df_test.iloc[row_i].text)
    gt_idxs = np.where(gt[row_i])[0]
    pred_idxs = np.where(pred[row_i])[0]
    gts = []
    
    for i in gt_idxs:
        l1 = label_to_emotion_text[idx2label[i]]
        gts += [l1]
    print("gt: ", ' '.join(gts))
    preds = []
    for i in pred_idxs:
        preds += [label_to_emotion_text[idx2label[i]]]
    print("pred: ", ' '.join(preds))
    
    preds_filtered = []
    if(len(preds) == 0):
        return ["null"]
    for i in preds:
        if not(i == study_emo):
             preds_filtered.append(i)
        
    return preds_filtered



In [118]:
study_emo = "disapproval"
print("label: ", emotion_text_to_label[study_emo])
study_idx = label2idx[emotion_text_to_label[study_emo]]
print("idx: ", study_idx)
mask = gt[:, study_idx] > 0
print("total: ", np.sum(mask))
got_wrong_mask = (gt[:, study_idx] != pred[:, study_idx]) * mask
print("got wrong total: ", np.sum(got_wrong_mask))
print("\n")
wrong_row_idxs = np.where(got_wrong_mask)[0]  

preds = []
for row_idx in wrong_row_idxs:
    print(row_idx)
    preds += probe(gt, pred, row_idx, study_emo)
    print("\n")

label:  10
idx:  22
total:  267
got wrong total:  172


10
[NAME] has towed the line of the Dark Side. He wouldn't cross it by doing something like this.
gt:  annoyance disapproval
pred:  neutral


15
I never wanted to punch osap harder after seeing that However not too hardly I cant afford them taking everything away
gt:  disapproval
pred:  


40
You didn't watch it to the end.
gt:  disapproval
pred:  neutral


51
Get back on your meds mate..You aren't funny at all...
gt:  annoyance disapproval
pred:  neutral annoyance


52
"But Aunt [NAME], I don't *want* to send Grandma back to Italy!"
gt:  neutral disapproval
pred:  neutral


75
Man I hate double standards. I love the show but change the word ‘white’ to ‘black’ and it’s suddenly not funny anymore
gt:  love disapproval
pred:  love


86
Not at all. I'm not going to argue with someone who doesn't understand what a strawman is.
gt:  disapproval
pred:  


91
Not really cringey. Its not really embarrassing for you i think people might ju

In [93]:
wrong_labels, wrong_counts = np.unique(preds, return_counts=True)
wrong_labels2 = []
wrong_counts2 = []
for k in np.argsort(wrong_counts)[::-1]:
    wrong_labels2 += [wrong_labels[k]]
    wrong_counts2 += [wrong_counts[k]]

In [94]:
ss = []
for l,c in zip(wrong_labels2, wrong_counts2):
    print(l, c)
    ss += ["{} ".format(l)]
   #  ss += ["{} ({}) ".format(l,c)]


wrong_freq_row = "\textbf{}  & "
wrong_freq_row += ", ".join(ss[:5])
wrong_freq_row + " \\ "

neutral 110
null 33
admiration 26
love 11
amusement 11
optimism 11
disapproval 10
joy 9
annoyance 8
curiosity 7
caring 5
disappointment 5
realization 4
gratitude 4
excitement 4
anger 3
confusion 3
surprise 2
sadness 2
remorse 2
desire 1
nervousness 1
relief 1
fear 1


'\textbf{}  & neutral , null , admiration , love , amusement  \\ '

In [62]:
y_true_study_idx = np.ones(np.sum(got_wrong_mask)) * study_idx
y_pred_study_idx = []
for row in pred[got_wrong_mask]:
    row_wrong_idxs = list(np.where(row)[0])
    row_wrongs = []
    for j in row_wrong_idxs:
        row_wrongs += [idx2label[j] ]
    
        
    y_pred_study_idx += []

wrong_labels, wrong_counts = np.unique(y_pred_study_idx, return_counts=True)
for k in np.argsort(wrong_counts)[::-1]:
#     print(label)
    print(label_to_emotion_text[wrong_labels[k]], wrong_counts[k])

In [129]:
df_test.iloc[1294]

text     Oh, it's definitely working and that worries m...
label                                                   19
Name: 1294, dtype: object